In [1]:
import numpy as np
import pandas as pd
import tracktor as tr
import sys
import cv2
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

In [7]:
#number of individuals in tracked video
n_inds = 8

# offset to be added based on toxtrac arena selection
offset_x = 370 # fish - 0; flume - 245; spider - 75; termite - 370; mouse - 85; tadpole - 45; zebrafish - 100
offset_y = 0 # fish - 0; flume - 0; spider - 0; termite - 0; mouse - 12; tadpole - 0; zebrafish - 0

# name of source video and paths
video = 'termite_video'
input_vidpath = '/mnt/ssd1/Documents/Vivek/tracktor/videos/' + video + '.mp4'
output_vidpath = '/mnt/ssd1/Documents/Vivek/tracktor/output/' + video + '_compared.mp4'
tracktor_datapath = '/mnt/ssd1/Documents/Vivek/tracktor/output/' + video + '_tracked.csv'
toxtrack_datapath = '/mnt/ssd1/Documents/Vivek/tracktor/output/' + video + '_toxtracked.txt'
idtrack_datapath = '/mnt/ssd1/Documents/Vivek/tracktor/output/' + video + '_idtracked.txt'
codec = 'DIVX' # try other codecs if the default doesn't work ('DIVX', 'avc1', 'XVID') note: this list is non-exhaustive
font = cv2.FONT_HERSHEY_SIMPLEX

In [8]:
## Open video
cap = cv2.VideoCapture(input_vidpath)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Open datafiles
df1 = pd.read_csv(tracktor_datapath)
df2 = pd.read_table(toxtrack_datapath, header=None, index_col=False, names=['frame', 'arena', 'id', 'pos_x', 'pos_y'])
df3 = pd.DataFrame({'pos_x':[], 'pos_y':[], 'id':[]})

df_tmp = pd.read_table(idtrack_datapath)
col_del = 'Unnamed: ' + str(n_inds*3)
df_tmp = df_tmp.drop(labels=col_del, axis=1)

for i in range(n_inds):
    tmp = df_tmp.iloc[:,3*i:3*(i+1)]
    tmp = tmp.rename(index=str, columns={'X'+str(i+1):'pos_x', 'Y'+str(i+1):'pos_y', 'ProbId'+str(i+1):'id'})
    tmp['id'] = i
    tmp['frame'] = np.arange(tmp.shape[0])
    df3 = pd.concat([df3, tmp])

In [9]:
## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap.read()[1].shape[1]),int(cap.read()[1].shape[0]))
out = cv2.VideoWriter(filename = output_vidpath, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)
last = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    this = cap.get(1)
    if ret == True:
        frame = cv2.resize(frame, None, fx = 1.0, fy = 1.0, interpolation = cv2.INTER_LINEAR)
        
        if df1[df1['frame'] == this].empty == False:
            x1 = df1[df1['frame'] == this]['pos_x']
            y1 = df1[df1['frame'] == this]['pos_y']
            for pts1 in range(0,len(x1)):
                cv2.circle(frame, (int(x1.values[pts1]),int(y1.values[pts1])), 7, (100,255,255), -1, cv2.LINE_AA)
        if df2[df2['frame'] == this].empty == False:
            x2 = df2[df2['frame'] == this]['pos_x']+offset_x
            y2 = df2[df2['frame'] == this]['pos_y']+offset_y
            for pts2 in range(0,len(x2)):
                cv2.circle(frame, (int(x2.values[pts2]),int(y2.values[pts2])), 5, (100,100,255), -1, cv2.LINE_AA)
        if df3[df3['frame'] == this].empty == False:
            x3 = df3[df3['frame'] == this]['pos_x']
            y3 = df3[df3['frame'] == this]['pos_y']
            for pts3 in range(0,len(x3)):
                if np.isnan(x3.values[pts3]) == False:
                    cv2.circle(frame, (int(x3.values[pts3]),int(y3.values[pts3])), 4, (255,100,100), -1, cv2.LINE_AA)
        
        # Add legend
        cv2.circle(frame, (20,20), 7, (100,255,255), -1, cv2.LINE_AA)
        cv2.putText(frame, '- Tracktor', (35,25), font, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.circle(frame, (20,50), 5, (100,100,255), -1, cv2.LINE_AA)
        cv2.putText(frame, '- ToxTrac', (35,55), font, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.circle(frame, (20,80), 4, (255,100,100), -1, cv2.LINE_AA)
        cv2.putText(frame, '- IDTracker', (35,85), font, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        # Display the resulting frame
        out.write(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 27:
            break
        
    if last == this:
        break
    
    last = this

# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [58]:
acc1 = len(df1['frame']) / (n_inds*this)
acc2 = len(df2['frame']) / (n_inds*this)
acc3 = (len(df3['frame']) - np.sum(np.isnan(df3['pos_x']))) / (n_inds*this)

print("--- toxtrac accuracy - %s ---" %acc1)
print("--- tracktor accuracy - %s ---" %acc2)
print("--- idtracker accuracy - %s ---" %acc3)

--- toxtrac accuracy - 0.78564 ---
--- tracktor accuracy - 0.9998666666666667 ---
--- idtracker accuracy - 0.9140933333333333 ---
